In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
independent=dataset[[ 'Age', 'EstimatedSalary','Gender_Male']]

In [10]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,True
1,35,20000,True
2,26,43000,False
3,27,57000,False
4,19,76000,True
...,...,...,...
395,46,41000,False
396,51,23000,True
397,50,20000,False
398,36,33000,True


In [11]:
dependent=dataset[['Purchased']]

In [12]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [14]:
#from sklearn.preprocessing import StandardScaler
#sc=StandardScaler()
#x_train=sc.fit_transform(x_train)
#x_test=sc.transform(x_test)

In [15]:
from sklearn.naive_bayes import ComplementNB
from sklearn. model_selection import GridSearchCV
param_grid = {'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
             'force_alpha':[True,False], 
             'fit_prior':[True,False], 
             'class_prior':[None],
             'norm':[True,False]}
grid = GridSearchCV(ComplementNB(),param_grid,refit= True,verbose = 3,n_jobs=-1,scoring='f1_weighted') 
grid.fit(x_train,y_train)  
print("Best parameters found: ", grid.best_params_)
y_pred = grid.predict(x_test)
print("Predictions: ", y_pred)

Fitting 5 folds for each of 80 candidates, totalling 400 fits
Best parameters found:  {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'norm': False}
Predictions:  [0 1 0 1 1 0 1 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0
 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 1 1 1
 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1]


C:\Users\user\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [50]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

                   

In [52]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [54]:
print(cm)

[[40 45]
 [20 29]]


In [56]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [58]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.67      0.47      0.55        85
           1       0.39      0.59      0.47        49

    accuracy                           0.51       134
   macro avg       0.53      0.53      0.51       134
weighted avg       0.57      0.51      0.52       134



In [60]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'alpha': 0.1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True, 'norm': False}: 0.5224047968734963


In [62]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.5834333733493396

In [64]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.021321,0.008962,0.041398,0.004202,0.1,None,True,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
1,0.021926,0.005923,0.036933,0.007166,0.1,None,True,True,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
2,0.020637,0.001630,0.034841,0.006285,0.1,None,True,False,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
3,0.019530,0.002511,0.028626,0.007103,0.1,None,True,False,False,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
4,0.016644,0.004474,0.028784,0.005580,0.1,None,False,True,True,"{'alpha': 0.1, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.014106,0.002558,0.025903,0.001112,1.0,None,True,False,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
76,0.014732,0.004657,0.027817,0.006388,1.0,None,False,True,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
77,0.014579,0.006415,0.023943,0.004003,1.0,None,False,True,False,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.509839,0.614012,0.563318,0.482156,0.556133,0.545092,0.045624,1
78,0.012983,0.004973,0.029480,0.004449,1.0,None,False,False,True,"{'alpha': 1.0, 'class_prior': None, 'fit_prior...",0.183156,0.172203,0.189203,0.189203,0.189203,0.184594,0.006623,41
